# LangChain starter kit

**[LangChain](https://docs.langchain.com/docs/)** is a framework for developing applications powered by LLMs (large language models).

In [1]:
import langchain
langchain.__version__

'0.3.4'

**[Components](https://python.langchain.com/docs/how_to/)** are the core building blocks you can use when building applications:

- **Prompt templates**

Prompt Templates are responsible for formatting user input into a format that can be passed to a language model.

- **Example selectors**

Example Selectors are responsible for selecting the correct few shot examples to pass to the prompt.

- **Chat models**

Chat Models are newer forms of language models that take messages in and output a message. 
> message in ➡️ message out

- **Messages**
  
Messages are the input and output of chat models. They have some content and a role, which describes the source of the message.

- **LLMs**

What LangChain calls LLMs are older forms of language models that take a string in and output a string.
> text in ➡️ text out

- **Output parsers**

Output Parsers are responsible for taking the output of an LLM and parsing into more structured format (e.g. prompt instructions + string parsing, Pydantic models...).

- **Document loaders**

Document Loaders are responsible for loading documents from a variety of sources.

- **Text splitters**

Text Splitters take a document and split into chunks that can be used for retrieval.
> documents in ➡️ chunks out

- **Embedding models**

Embedding Models take a piece of text and create a numerical representation of it. 
> text in ➡️ vector out

- **Vector stores**
    - [Pinecone](https://www.pinecone.io/)
    - [Weaviate](https://weaviate.io/)
    - [Chroma](https://www.trychroma.com/)
    - [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)

Vector Stores are databases that can efficiently store and retrieve embeddings.

- **Retrievers**

Retrievers are responsible for taking a query and returning relevant documents.

- **Tools**

LangChain Tools contain a description of the tool (to pass to the language model) as well as the implementation of the function to call. 

- ⛓️ **Chains**

One point about LangChain Expression Language is that any two runnables can be "chained" together into sequences. The output of the previous runnable's ```.invoke()``` call is passed as input to the next runnable. This can be done using the pipe operator ( **|** ).

- 🤖 **Agents** 

By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

In [2]:
import json
import subprocess
keys = json.loads(subprocess.run(["sops", "decrypt", "API_keys.enc.json"], capture_output=True).stdout)

In [3]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

from langchain_openai import ChatOpenAI
from langchain_openai.llms import OpenAI

from langchain_ollama import ChatOllama
from langchain_ollama.llms import OllamaLLM

In [4]:
messages = [
    SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
    HumanMessage(content="I like tomatoes, what should I eat?")
]

Running remotely through OpenAI API

In [5]:
%%time
chat_openai = ChatOpenAI(temperature=.7, openai_api_key=keys['OpenAI'])
output = chat_openai.invoke(messages)
print (type(output))
print ("-----")
print (output.content)
print ("_____")

<class 'langchain_core.messages.ai.AIMessage'>
-----
You might enjoy a Caprese salad with fresh tomatoes, mozzarella, basil, and balsamic glaze.
_____
CPU times: user 121 ms, sys: 7.96 ms, total: 129 ms
Wall time: 1.1 s


In [6]:
%%time
model_openai = OpenAI(openai_api_key=keys['OpenAI'])
output = model_openai.invoke(messages)
print (type(output))
print ("-----")
print (output)
print ("_____")

<class 'str'>
-----

System: You could try a tomato-based dish like pasta with marinara sauce or a caprese salad.
_____
CPU times: user 98.1 ms, sys: 4.19 ms, total: 102 ms
Wall time: 1.09 s


Running locally using Ollama


In [7]:
%%time
chat_ollama = ChatOllama(model="llama3.2")
output = chat_ollama.invoke(messages)
print (type(output))
print ("-----")
print (output.content)
print ("_____")

<class 'langchain_core.messages.ai.AIMessage'>
-----
How about a delicious pasta with homemade tomato sauce and fresh mozzarella for a satisfying meal?
_____
CPU times: user 75.6 ms, sys: 2.62 ms, total: 78.2 ms
Wall time: 3.55 s


In [8]:
%%time
model_ollama = OllamaLLM(model="llama3.2")
output = model_ollama.invoke(messages)
print (type(output))
print ("-----")
print (output)
print ("_____")

<class 'str'>
-----
You can make a delicious Caprese salad with fresh mozzarella, basil, and juicy tomatoes, all dressed with olive oil and balsamic vinegar!
_____
CPU times: user 75.8 ms, sys: 1.98 ms, total: 77.8 ms
Wall time: 5.38 s


In [9]:
import requests
from langchain_core.tools import tool

@tool
def convert(currency_in: str, currency_out: str, amount: float) -> str:
    """Convert an amount between two currencies.
    Args:
      currency_in (str): The current currency eg USD
      currency_out (str): The target currency eg EUR
      amount (float): The amount to convert.
    """
    url = 'https://v6.exchangerate-api.com/v6/%s/pair/%s/%s'%(keys['ExchangeRate'],currency_in,currency_out)
    response = requests.get(url)
    data = response.json()
    return json.dumps(float(amount)*data['conversion_rate'])

In [10]:
messages= [
    SystemMessage(content="You are an helpful AI bot."),
    HumanMessage(content="How much is $100 dollar to euro?")
]

In [11]:
chat_openai = ChatOpenAI(temperature=.7, openai_api_key=keys['OpenAI'])
chat_openai_with_tools = chat_openai.bind_tools([convert])
output = chat_openai_with_tools.invoke(messages)
messages.append(output)

In [12]:
for tool in output.additional_kwargs["tool_calls"]:
    selected_tool = {"convert": convert}[tool["function"]["name"].lower()]
    resp = selected_tool.invoke(json.loads(tool["function"]["arguments"]))
    messages.append({"role": "tool", "content":resp, "tool_call_id": tool["id"]})

In [13]:
messages

[SystemMessage(content='You are an helpful AI bot.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How much is $100 dollar to euro?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wt4VX5DLVrYGzod8qKvsBvGL', 'function': {'arguments': '{"currency_in":"USD","currency_out":"EUR","amount":100}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 107, 'total_tokens': 130, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-94ab557e-0109-4576-8344-5fbe2f935c08-0', tool_calls=[{'name': 'convert', 'args': {'currency_in': 'USD', 'currency_out': 'EUR',

In [14]:
output = chat_openai_with_tools.invoke(messages)
print (output.content)

$100 is equivalent to €94.45.


In [15]:
messages= [
    SystemMessage(content="You are an helpful AI bot"),
    HumanMessage(content="What is 100 us dollars in euros?")
]

In [16]:
chat_ollama = ChatOllama(model="llama3.2")
chat_ollama_with_tools = chat_ollama.bind_tools([convert])
output = chat_ollama_with_tools.invoke(messages)
messages.append(output)

In [17]:
output

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-11-20T14:41:56.337977097Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'convert', 'arguments': {'amount': '100', 'currency_in': 'USD', 'currency_out': 'EUR'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 15800891848, 'load_duration': 19635276, 'prompt_eval_count': 231, 'prompt_eval_duration': 12065589000, 'eval_count': 30, 'eval_duration': 3671301000}, id='run-a7838497-dc43-4996-b1b3-1d8b67f8fbd9-0', tool_calls=[{'name': 'convert', 'args': {'amount': '100', 'currency_in': 'USD', 'currency_out': 'EUR'}, 'id': 'd8f835eb-50fd-4819-ac03-d73052288413', 'type': 'tool_call'}], usage_metadata={'input_tokens': 231, 'output_tokens': 30, 'total_tokens': 261})

In [18]:
for tool in output.tool_calls:
    selected_tool = {"convert": convert}[tool["name"].lower()]
    resp = selected_tool.invoke(tool["args"])
    messages.append({"role": "tool", "content":resp, "tool_call_id": tool["id"]})

In [19]:
messages

[SystemMessage(content='You are an helpful AI bot', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 100 us dollars in euros?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-11-20T14:41:56.337977097Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'convert', 'arguments': {'amount': '100', 'currency_in': 'USD', 'currency_out': 'EUR'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 15800891848, 'load_duration': 19635276, 'prompt_eval_count': 231, 'prompt_eval_duration': 12065589000, 'eval_count': 30, 'eval_duration': 3671301000}, id='run-a7838497-dc43-4996-b1b3-1d8b67f8fbd9-0', tool_calls=[{'name': 'convert', 'args': {'amount': '100', 'currency_in': 'USD', 'currency_out': 'EUR'}, 'id': 'd8f835eb-50fd-4819-ac03-d73052288413', 'type': 'tool_call'}], usage_metadata={'input_tokens': 231, 'output_tokens': 30, 'tot

In [20]:
output = chat_ollama_with_tools.invoke(messages)
print (output.content)

The current exchange rate is approximately 1 USD = 0.8825 EUR.

To convert $100 USD to euros, we can multiply:

$100 x 0.8825 = €94.45

So, $100 USD is equivalent to approximately €94.45 EUR.


In [21]:
text = "A text embedding is a piece of text projected into a high-dimensional latent space." 

In [22]:
from langchain_openai import OpenAIEmbeddings
embeddings_openai = OpenAIEmbeddings(openai_api_key=keys['OpenAI'])

In [23]:
text_embedding_openai = embeddings_openai.embed_query(text)
print (f"Here's a sample: {text_embedding_openai[:5]}...")
print (f"Your embedding is length {len(text_embedding_openai)}")

Here's a sample: [-0.02410108782351017, 0.004306534770876169, -0.006926639936864376, -0.0012955142883583903, 0.01679968647658825]...
Your embedding is length 1536


In [24]:
from langchain_ollama import OllamaEmbeddings
embeddings_ollama = OllamaEmbeddings(model="nomic-embed-text")

In [25]:
text_embedding_ollama = embeddings_ollama.embed_query(text)
print (f"Here's a sample: {text_embedding_ollama[:5]}...")
print (f"Your embedding is length {len(text_embedding_ollama)}")

Here's a sample: [0.042513944, 0.07905781, -0.16957015, -0.10675651, -0.012025197]...
Your embedding is length 768


In [26]:
from langchain.prompts import PromptTemplate

template = """
I really want to travel to {location}. What should I do there?

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

final_prompt = prompt.format(location='Rome')

In [27]:
print (chat_ollama.invoke(final_prompt).content)

Rome offers a plethora of attractions, including the Colosseum, Vatican City (including the Sistine Chapel and St. Peter's Basilica), and indulging in delicious Italian cuisine like pizza and gelato.


In [28]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.prompts import FewShotPromptTemplate
from langchain.vectorstores import Chroma

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

examples = [
    {"input": "blue", "output": "sky"},
    {"input": "green", "output": "grass"},
    {"input": "red", "output": "tomato"},
    {"input": "orange", "output": "basketball"},
    {"input": "yellow", "output": "banana"},
]

In [29]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, 
    OllamaEmbeddings(model="nomic-embed-text"),
    Chroma, 
    k=2
)

In [30]:
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Find an object that often matches the provided color.",
    suffix="Input: {noun}\nOutput:",
    input_variables=["noun"],
)

In [31]:
my_noun = "purple"
print(similar_prompt.format(noun=my_noun))

Find an object that often matches the provided color.

Example Input: blue
Example Output: sky

Example Input: orange
Example Output: basketball

Input: purple
Output:


In [32]:
chat_ollama = ChatOllama(model="llama3.2")
print (chat_ollama.invoke(similar_prompt.format(noun=my_noun)).content)

grapes


In [33]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="good_string", description="This is your response, a reformatted response.")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"good_string": string  // This is your response, a reformatted response.
}
```


In [34]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

In [35]:
chat_ollama = ChatOllama(model="llama3.2")
chain = prompt | chat_ollama | output_parser

In [36]:
print (chain.invoke({"user_input": "welcom to califonya!"}))

{'good_string': 'Welcome to California!'}


In [37]:
from langchain.document_loaders import UnstructuredURLLoader
urls = [ 
    "http://www.paulgraham.com/quo.html"
]
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()
print (data[0].page_content[:500])

"The less confident you are, the more serious you have to act."  Tara Ploughman "The condition of man is already close to satiety and arrogance, and there is danger of destruction of everything in existence."  a Brahmin to Onesicritus, 327 BC, reported in Strabo's Geography "Change breaks the brittle."  Jan Houtema The sons of Hermes love to play, And only do their best when they Are told they oughtn't; Apollo's children never shrink From boring jobs but have to think Their work important.  


In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 150,
    chunk_overlap  = 30,
)

In [39]:
texts = text_splitter.split_documents(data)
print (f"You have {len(texts)} documents")
print ("_____")
print (texts[0].page_content, "\n")
print (texts[1].page_content)

You have 235 documents
_____
"The less confident you are, the more serious you have to act."  Tara Ploughman "The condition of man is already close to satiety and arrogance, and 

to satiety and arrogance, and there is danger of destruction of everything in existence."  a Brahmin to Onesicritus, 327 BC, reported in Strabo's


In [40]:
%env USER_AGENT='Mozilla/5.0 (X11; Linux i686; rv:110.0) Gecko/20100101 Firefox/110.0.'

env: USER_AGENT='Mozilla/5.0 (X11; Linux i686; rv:110.0) Gecko/20100101 Firefox/110.0.'


In [41]:
from langchain.document_loaders import WebBaseLoader
urls = [
    "https://magazine.sebastianraschka.com/p/llm-training-rlhf-and-its-alternatives",
    "https://www.tensorops.ai/post/what-is-mixture-of-experts-llm",
    "https://medium.com/@b.terryjack/deep-learning-the-transformer-9ae5e9c5a190"
]
loader = WebBaseLoader(urls)
data = loader.load()
print (data[0].metadata)

{'source': 'https://magazine.sebastianraschka.com/p/llm-training-rlhf-and-its-alternatives', 'title': 'LLM Training: RLHF and Its Alternatives', 'description': 'I frequently reference a process called Reinforcement Learning with Human Feedback (RLHF) when discussing LLMs, whether in the research news or tutorials.', 'language': 'en'}


In [42]:
from docling.document_converter import DocumentConverter
result = DocumentConverter().convert("https://arxiv.org/pdf/1706.03762")

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


ERR#: COULD NOT CONVERT TO RS THIS TABLE TO COMPUTE SPANS


In [43]:
texts = text_splitter.create_documents([result.document.export_to_markdown()])
print (f"You have {len(texts)} documents")
print ("_____")
print (texts[0].page_content[:500])

You have 457 documents
_____
Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in


In [44]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings_ollama)
retriever = db.as_retriever()

In [45]:
retriever.invoke("What is an attention mechanism?")

[Document(metadata={}, page_content='Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in various tasks, allowing modeling of'),
 Document(metadata={}, page_content='Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a'),
 Document(metadata={}, page_content='An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are'),
 Document(metadata={}, page_content='The two most commonly used attention functions are additive attention [2], and dot-product (multiplicative) attention. Dot-product attention is')]

In [46]:
from langchain_core.output_parsers import StrOutputParser
chat_ollama = ChatOllama(model="llama3.2")

In [47]:
template = """Your role is to give a dish typical of the city or region that the user has chosen. Return the name of the dish only and nothing else.  
Location: {location}
Dish: 
"""
part1 = PromptTemplate(input_variables=["location"], template=template) | chat_ollama | StrOutputParser()

In [48]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
Meal: {meal}
Recipe: 
"""
part2 = PromptTemplate(input_variables=["meal"], template=template) | chat_ollama | StrOutputParser()

In [49]:
from langchain.schema.runnable import RunnablePassthrough
chain = (
    {"meal": part1} 
    | RunnablePassthrough.assign(recipe=part2)
)

In [50]:
result = chain.invoke({"location": "Chicago"})

In [51]:
import textwrap
wrap_width = 80
print("Output:\n_____")
for key, value in result.items():
    print(f"{key.capitalize()}:")
    paragraphs = value.split('\n\n')
    for paragraph in paragraphs:
        wrapped_paragraph = textwrap.fill(paragraph, width=wrap_width)
        print(wrapped_paragraph)
        print()

Output:
_____
Meal:
Deep-Dish Pizza

Recipe:
Deep-Dish Pizza Recipe:

Ingredients:

* 1 lb pizza dough (homemade or store-bought) * 2 cups shredded mozzarella
cheese * 1 cup shredded cheddar cheese * 1/4 cup chopped pepperoni * 1/4 cup
chopped onion * 3 cloves garlic, minced * 1 can (28 oz) crushed tomatoes * 1 tsp
dried oregano * Salt and pepper to taste

Instructions:

1. Preheat oven to 425°F (220°C). 2. Roll out pizza dough into a thick circle,
about 1/4 inch thick. 3. Transfer the dough to a deep-dish pizza pan or round
cake pan. 4. Spread crushed tomatoes evenly over the dough, leaving a 1-inch
border around the edges. 5. Sprinkle mozzarella and cheddar cheese over the
tomato sauce. 6. Top with pepperoni, onion, and garlic. 7. Bake for 25-30
minutes or until crust is golden brown and cheese is melted and bubbly. 8.
Remove from oven and let cool for a few minutes before serving.

Enjoy your homemade deep-dish pizza!

